In [3]:
# Importing Libraries
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from elasticsearch import helpers

In [4]:
# Establishing client with elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [5]:
# Reading csv into a pandas dataframe
df = pd.read_csv("Downloads\hospital_directory_sample.csv")
df.dropna(subset=['Location_Coordinates'], inplace=True)
df.fillna("Not Available", inplace=True)

In [6]:
mapping = {
  "settings": {
    "number_of_shards": 2,
    "number_of_replicas": 1
  },
   "mappings": {
    "dynamic": "true",
    "_source": {
      "enabled": "true"
    },
    "properties": {
      "Location_Coordinates": {
        "type": "geo_point"
      }
    }
  }
}


In [7]:
# Create index with mappings, skips if exists
try:
    es.indices.create(index="hosp", body=mapping)
    print("New Index Created")
except Exception as e:
    print("Index Already Exists")

Index Already Exists


In [8]:
def index_batch(docs):
    """
    helper function used to batch index docs to es
    """
    requests = []
    for i, doc in enumerate(docs):
        request = doc
        request["_op_type"] = "index"
        request["_index"] = "hosp"
        requests.append(request)
    bulk(es, requests)

count = 0
hosp_lst = []
for i in df.iterrows():
    index = {
        'Sr_No': i[1][0],
        'Location_Coordinates': i[1][1],
        'Location': i[1][2],
        'Hospital_Name': i[1][3],
        'Hospital_Category': i[1][4],
        'Hospital_Care_Type': i[1][5],
        'Discipline_Systems_of_Medicine': i[1][6],
        'Address_Original_First_Line': i[1][7],
        'State': i[1][8],
        'District': i[1][9],
        'Subdistrict': i[1][10],
        'Pincode': i[1][11],
        'Telephone': i[1][12],
        'Mobile_Number': i[1][13],
        'Emergency_Num': i[1][14],
        'Ambulance_Phone_No': i[1][15],
        'Bloodbank_Phone_No': i[1][16],
        'Foreign_pcare': i[1][17],
        'Tollfree': i[1][18],
        'Helpline': i[1][19],
        'Hospital_Fax': i[1][20],
        'Hospital_Primary_Email_Id': i[1][21],
        'Hospital_Secondary_Email_Id': i[1][22],
        'Website': i[1][23],
        'Specialties': i[1][24],
        'Facilities': i[1][25],
        'Accreditation': i[1][26],
        'Hospital_Regis_Number': i[1][27],
        'Registeration_Number_Scan': i[1][28],
        'Nodal_Person_Info': i[1][29],
        'Nodal_Person_Tele': i[1][30],
        'Nodal_Person_Email_Id': i[1][31],
        'Town': i[1][32],
        'Subtown': i[1][33],
        'Village': i[1][34],
        'Establised_Year': i[1][35],
        'Ayush': i[1][36],
        'Miscellaneous_Facilities': i[1][37],
        'Number_Doctor': i[1][38],
        'Num_Mediconsultant_or_Expert': i[1][39],
        'Total_Num_Beds': i[1][40],
        'Number_Private_Wards': i[1][41],
        'Num_Bed_for_Eco_Weaker_Sec': i[1][42],
        'Empanelment_or_Collaboration_with': i[1][43],
        'Emergency_Services': i[1][44],
        'Tariff_Range': i[1][45],
        'State_ID': i[1][46],
        'District_ID': i[1][47]
    }
    hosp_lst.append(index)
    count += 1
    index_batch(hosp_lst)

print("Indexed {} documents.".format(count))
es.indices.refresh(index="hosp")
print("Done indexing.")

In [51]:
count = 0
hosp_lst = []
for i in df.iterrows():
    index = {
        'Sr_No': i[1][0],
        'Location_Coordinates': i[1][1],
        'Location': i[1][2],
        'Hospital_Name': i[1][3],
        'District': i[1][9]
    }
    hosp_lst.append(index)
    count += 1
    index_batch(hosp_lst)

print("Indexed {} documents.".format(count))
es.indices.refresh(index="hosp")

Indexed 374 documents.


{'_shards': {'total': 4, 'successful': 2, 'failed': 0}}

In [57]:
# Searching hospitals in elasticsearch by coordinates
res = es.search(index="hosp", body={
    "query": {
        "match":{
            "District":"Anantapur"
            
            }
        }
})
for hit in res["hits"]["hits"]:
    print(hit)

    

{'_index': 'hosp', '_type': '_doc', '_id': 'OluK33IB3RdC-qdPC4Lu', '_score': 1.8441103, '_source': {'Sr_No': 7, 'Location_Coordinates': '14.6795736, 77.592469', 'Location': '7/201, Court Road Anantapur 515001  Anantapur', 'Hospital_Name': 'Aasha Hospitals', 'District': 'Anantapur'}}
{'_index': 'hosp', '_type': '_doc', '_id': 'QVuK33IB3RdC-qdPDILe', '_score': 1.8441103, '_source': {'Sr_No': 7, 'Location_Coordinates': '14.6795736, 77.592469', 'Location': '7/201, Court Road Anantapur 515001  Anantapur', 'Hospital_Name': 'Aasha Hospitals', 'District': 'Anantapur'}}
{'_index': 'hosp', '_type': '_doc', '_id': 'QluK33IB3RdC-qdPDILe', '_score': 1.8441103, '_source': {'Sr_No': 8, 'Location_Coordinates': '10.123, 10.456', 'Location': '10-470, Adimurthy Nagar, Opposite Guild Off Service School', 'Hospital_Name': 'Abhaya Kidney Care Multispeciality Hospital', 'District': 'Anantapur'}}
{'_index': 'hosp', '_type': '_doc', '_id': 'SluK33IB3RdC-qdPDoIh', '_score': 1.8441103, '_source': {'Sr_No': 8, 'L